So here's what you should do. Pick a dataset. It could be one you've worked with before or it could be a new one. Then build the best decision tree you can.

Now try to match that with the simplest random forest you can. For our purposes measure simplicity with runtime. Compare that to the runtime of the decision tree. This is imperfect but just go with it.

In [23]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import tree
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt

%matplotlib inline

# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

C:\Users\sgfis\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [26]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [27]:
#dropping Columns with over 50% null values to start
drop = ['dti_joint', 'annual_inc_joint', 'verification_status_joint', 
        'il_util', 'mths_since_rcnt_il', 'open_acc_6m', 'inq_last_12m', 
       'total_cu_tl', 'inq_fi', 'all_util', 'max_bal_bc', 'open_rv_24m',
       'open_rv_12m', 'total_bal_il', 'open_il_24m', 'open_il_12m', 
        'open_il_6m', 'mths_since_last_record', 'mths_since_recent_revol_delinq']

y2015 = y2015.drop(columns = drop)

In [28]:
# dropping redundant and not useful categorical columns with high variety
drop1 = ['id', 'member_id']

y2015 =  y2015.drop(columns = drop1)

In [29]:
# Fixing multicollinearity
drop2 = ['funded_amnt', 'funded_amnt_inv', 'installment']

y2015 = y2015.drop(columns = drop2)

In [30]:
y2015 = y2015.dropna()

In [31]:
useless_drops = ['issue_d', 'title', 'initial_list_status', 'last_pymnt_d', 'next_pymnt_d',
                 'last_credit_pull_d', 'application_type']

y2015  = y2015.drop(columns = useless_drops)

In [32]:
challenge_drop = ['last_pymnt_amnt', 'out_prncp']
y2015 = y2015.drop(columns=challenge_drop)

In [33]:
y2015 = pd.get_dummies(y2015, drop_first = True)

In [34]:
loanstatus = ['loan_status_Default', 'loan_status_In Grace Period', 'loan_status_Late (16-30 days)',
             'loan_status_Late (31-120 days)']

In [35]:
X = y2015.drop(loanstatus, 1)
Y = y2015[loanstatus]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

# Decision Tree

In [41]:
import time
from sklearn import metrics

start_time = time.time()

decision_tree = tree.DecisionTreeClassifier(
    criterion='entropy',
    max_features=1,
    max_depth=10,
    random_state = 1337
)
decision_tree.fit(X_train, y_train)

preddt = decision_tree.predict(X_test)

print(cross_val_score(decision_tree, X, Y, cv=10))
print(preddt)
print("Accuracy:",metrics.accuracy_score(y_test, preddt))
print("--- %s seconds ---" % (time.time() - start_time))

[0.93207856 0.9375341  0.93726132 0.93316967 0.93642565 0.92933151
 0.93124147 0.92905866 0.92960437 0.92987722]
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 ...
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
Accuracy: 0.928931932887737
--- 0.791881799697876 seconds ---


# Random Forest

In [22]:
start_time = time.time()

rfc = ensemble.RandomForestClassifier()

print(cross_val_score(rfc, X, Y, cv=10))

print("--- %s seconds ---" % (time.time() - start_time))

C:\Users\sgfis\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\sgfis\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\sgfis\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\sgfis\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\sgfis\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: F

[0.93344244 0.9386252  0.9375341  0.93426077 0.93642565 0.93042292
 0.93206003 0.93069577 0.93015007 0.93206003]
--- 20.687072038650513 seconds ---
